$
\newcommand\op{\operatorname}
\newcommand\var{\op{var}}
\newcommand\cov{\op{cov}}
\newcommand\Normal{\op{Normal}}
\newcommand\MvNormal{\op{MvNormal}}
$

https://escholarship.org/content/qt0j50v7hx/qt0j50v7hx_noSplash_f8d003538b36370a6a9c11bfe39ae39a.pdf で取り上げられているモデルは

$$
\begin{aligned}
&
Q = b_1 P + d_1 I + U_1,
\\ &
P = b_2 Q + d_2 W + U_2,
\\ &
\text{$\{I,W\}$ and $\{U_1,U_2\}$ are independent},
\\ &
\begin{bmatrix}
E[U_1] \\
E[U_2] \\
\end{bmatrix} = \mu,
\quad
\begin{bmatrix}
\var(U_1) & \cov(U_1,U_2) \\
\cov(U_2,U_1) & \var(U_2) \\
\end{bmatrix} = \Sigma.
\end{aligned}
$$

この条件のうち前者の2つは次と同値である:

$$
Q = \frac{d_1 I + U_1 + b_1(d_2 W + U_2)}{1 - b_1 b_2}, \quad
P = \frac{d_2 W + U_2 + b_2(d_1 I + U_1)}{1 - b_1 b_2}.
$$

このように生成したデータの数値からパラメータ値 $b_1,b_2,d_1,d_2,\mu,\Sigma$ を推定する方法を考える.

$$
\begin{bmatrix}
A & C \\
B & D \\
\end{bmatrix}
=
T_{11}^{-1} T_{12},
\quad
T_{11}
=
\begin{bmatrix}
\var(I)   & \cov(I,W) \\
\cov(W,I) & \var(W) \\
\end{bmatrix},
\quad
T_{12}
=
\begin{bmatrix}
\cov(I,Q) & \cov(I,P) \\
\cov(W,Q) & \cov(W,P) \\
\end{bmatrix},
$$

とおくと,

$$
A = \frac{d_1}{1 - b_1 b_2}, \quad
B = \frac{b_1 d_2}{1 - b_1 b_2}, \quad
C = \frac{b_2 d_1}{1 - b_1 b_2}, \quad
D = \frac{d_2}{1 - b_1 b_2}.
$$

すなわち,

$$
b_1 = B/D, \quad
b_2 = C/A, \quad
d_1 = (1 - b_1 b_2)A, \quad
d_2 = (1 - b_1 b_2)D.
$$

そして,

$$
U_1 = Q - (b_1 P + d_1 I), \quad
U_2 = P - (b_2 Q + d_2 W).
$$

もしくは,

$$
V_1 = Q - (AI + BW), \quad
V_2 = P - (CI + DW)
$$

とおくと,

$$
U_1 = V_1 - b_1 V_2, \quad
U_2 = V_2 - b_2 V_1, \quad
\mu =
\begin{bmatrix}
E[U_1] \\
E[U_2] \\
\end{bmatrix},
\quad
\Sigma =
\begin{bmatrix}
\var(U_1) & \cov(U_1,U_2) \\
\cov(U_2,U_1) & \var(U_2) \\
\end{bmatrix}.
$$

データの数値に対して以上の手続きを適用すれば, パラメータ値 $b_1,b_2,d_1,d_2,\mu,\Sigma$ を推定できる.

In [1]:
using Distributions
using Random
#using StatsPlots
#default(fmt=:png, titlefontsize=8, tickfontsize=6, guidefontsize=8, legendfontsize=8)

In [2]:
function rand_qpiw(b1, b2, d1, d2, s1, s2)
    u1, u2, i, w = randn(), randn(), randn(), randn()
    q0 = d1*i + s1*u1
    p0 = d2*w + s2*u2
    q = (q0 + b1*p0)/(1 - b1*b2)
    p = (p0 + b2*q0)/(1 - b1*b2)
    q, p, i, w
end

function rand_qpiw!(b1, b2, d1, d2, s1, s2, qpiw::Matrix)
    Threads.@threads for i in axes(qpiw, 2)
        qpiw[:, i] .= rand_qpiw(b1, b2, d1, d2, s1, s2)
    end
    qpiw
end

function rand_qpiw!(b1, b2, d1, d2, s1, s2, n::Integer)
    qpiw = Matrix{Float64}(undef, 4, n)
    rand_qpiw!(b1, b2, d1, d2, s1, s2, qpiw)
end

rand_qpiw! (generic function with 2 methods)

In [3]:
b1, b2, d1, d2, s1, s2 = -1, 1, 0.4, 0.6, √0.5, √0.8
n = 10^6
qpiw = rand_qpiw!(b1, b2, d1, d2, s1, s2, n)
q, p, i, w = eachrow(qpiw);

In [4]:
X2 = [i w]
y2 = [q p]

β̂2 = X2 \ y2

2×2 Matrix{Float64}:
  0.200217  0.198295
 -0.300601  0.300411

In [5]:
[
       d1 b2*d1
    b1*d2    d2
]/(1 - b1*b2)

2×2 Matrix{Float64}:
  0.2  0.2
 -0.3  0.3

In [6]:
ww, xx, yy, zz = β̂2
bb1 = xx/zz
bb2 = yy/ww
dd1 = ww * (1 - bb1*bb2)
dd2 = zz * (1 - bb1*bb2)
[bb1, bb2, dd1, dd2]

4-element Vector{Float64}:
 -1.000630469293369
  0.9904022542034342
  0.39863743287608594
  0.5981269190769208

In [7]:
u1 = q - (bb1*p + dd1*i)
u2 = p - (bb1*q + dd2*w)
fit(MvNormal, [u1 u2]')

FullNormal(
dim: 2
μ: [-0.00025276663730480325, 0.0008226382410363626]
Σ: [0.5003230872667552 0.500322981735782; 0.500322981735782 1.0176682302929931]
)


In [8]:
mvn = fit(MvNormal, [i w q p]')
@show μ = mean(mvn)
Σ = cov(mvn)

μ = mean(mvn) = [0.002528178306651554, -0.00011267079764789343, 0.0005257053910915815, 0.00022920997177126542]


4×4 Matrix{Float64}:
 1.00166       0.000896279   0.20028    0.198894
 0.000896279   1.00108      -0.300746   0.300914
 0.20028      -0.300746      0.455156  -0.125455
 0.198894      0.300914     -0.125455   0.454836

In [9]:
Σ11 = Σ[1:2, 1:2]
Σ12 = Σ[1:2, 3:4]
Σ21 = Σ[3:4, 1:2]
Σ22 = Σ[3:4, 3:4]
β̂ = Σ11 \ Σ12

2×2 Matrix{Float64}:
  0.200217  0.198296
 -0.300601  0.300411

In [10]:
X = [i w]
q̂, p̂ = eachcol(X*β̂)
v1, v2 = q - q̂, p - p̂
u1, u2 = v1 - bb1*v2, v2 - bb2*v1
mvn = fit(MvNormal, [u1 u2]')
@show mean(mvn)
cov(mvn)

mean(mvn) = [-0.0002527682535713289, -0.00022405982824296974]


2×2 Matrix{Float64}:
 0.500323    0.00299583
 0.00299583  0.791656

In [11]:
ww, xx, yy, zz = β̂
bb1 = xx/zz
bb2 = yy/ww
dd1 = ww * (1 - bb1*bb2)
dd2 = zz * (1 - bb1*bb2)
[bb1, bb2, dd1, dd2]

4-element Vector{Float64}:
 -1.0006305658938555
  0.9904050788635844
  0.3986380900391199
  0.5981277424478418

In [12]:
u1 = q - (bb1*p + dd1*i)
u2 = p - (bb2*q + dd2*w)
mvn = fit(MvNormal, [u1'; u2'])
@show mean(mvn)
cov(mvn)

mean(mvn) = [-0.00025276827658837265, -0.00022405978771487714]


2×2 Matrix{Float64}:
 0.500323    0.00299517
 0.00299517  0.791659